In [ ]:
using Revise
using DrWatson
@quickactivate "SpikingNeuralNetworks"
using SpikingNeuralNetworks
SNN.@load_units
import SpikingNeuralNetworks: AdExParameter, IFParameter, IFConstParameter, AdExConstParameter
using Statistics, Random
using Plots

In [ ]:
## Neuron parameters
τm = 20ms
C = 300pF # Capacitance
R = τm / C

## Neuron parameters
τre = 1ms # Rise time for excitatory synapses
τde = 6ms # Decay time for excitatory synapses
τri = 0.5ms # Rise time for inhibitory synapses 
τdi = 2ms # Decay time for inhibitory synapses

# Input and synapse paramater
N = 1000
νe = 4.5Hz # Rate of external input to E neurons 
νi = 2.25Hz # Rate of external input to I neurons 
p_in = 1.0 #1.0 # 0.5 
σ_in_E = 1.78pF

σEE = 2.76pF # Initial E to E synaptic weight
σIE = 48.7pF # Initial I to E synaptic weight
σEI = 1.27pF # Synaptic weight from E to I
σII = 16.2pF # Synaptic weight from I to I

Random.seed!(23)
duration = 2000ms # 2 seconds
pltdur = 200e1# 100e1

In [ ]:
LKD_AdEx_exc = 
    AdExParameter(τm = 20ms, Vt = -52mV, Vr = -60mV, El = -70mV, R = 20ms/300pF, ΔT = 2mV, τw = 150ms, a = 4nS,
    b = 0.805pA, τabs = 1ms, τre = τre, τde = τde, τri = τri, τdi = τdi, E_i = -75mV, E_e = 0mV, At = 10mV, τT = 30ms)

LKD_IF_inh =
    IFParameter(τm = 20ms, Vt = -52mV, Vr = -60mV, El = -62mV, R = 20ms/300pF, ΔT = 2mV, 
    τre = τre, τde = τde, τri = τri, τdi = τdi, E_i = -75mV, E_e = 0mV, τabs = 1ms)

## Exploring weights

In [ ]:
νpre = 20Hz # Rate of external input to presynaptic neuron
νpost = 4.5Hz # Rate of external input to postsynaptic neuron

Pre = SNN.AdEx(; N = 1, param = LKD_AdEx_exc)
Post = SNN.IF(; N = 1, param = LKD_IF_inh) 

In [ ]:
Input_Pre = SNN.Poisson(; N = 100, param = SNN.PoissonParameter(; rate = νpre))
ProjPre = SNN.SpikingSynapse(Input_Pre, Pre, :ge; σ = σ_in_E, p = p_in) # connection from input to presynaptic neuron
Input_Post = SNN.Poisson(; N = 100, param = SNN.PoissonParameter(; rate = νpost))
ProjPost = SNN.SpikingSynapse(Input_Post, Post, :ge; σ = σ_in_E, p = p_in) # connection from input to postsynaptic neuron

In [ ]:
function average_firing_rate(E, I, bin_width = 10) # in milliseconds
    num_bins = Int(length(E.records[:fire]) / bin_width)
    bin_edges = 1:bin_width:(num_bins * bin_width)
    # avg spikes at each time step
    E_neuron_spikes = map(sum, E.records[:fire]) ./ E.N
    I_neuron_spikes = map(sum, I.records[:fire]) ./ E.N
    # Count the number of spikes in each bin
    E_bin_count = [sum(E_neuron_spikes[i:i+bin_width-1]) for i in bin_edges]
    I_bin_count = [sum(I_neuron_spikes[i:i+bin_width-1]) for i in bin_edges]
    return mean(E_bin_count), mean(I_bin_count)
end

### Before connecting neurons

In [ ]:
P = [Pre, Post, Input_Pre, Input_Post]
C = [ProjPre, ProjPost]

SNN.monitor([Pre, Post], [:v, :fire]) 
SNN.sim!(P, C; duration = duration)

In [ ]:
size(Pre.records[:fire])

In [ ]:
plot(SNN.vecplot(Pre, :v),
xlabel = "Time (ms)", 
ylabel = "Membrane Potential (mV)", 
title = "Presynaptic neuron with Poisson input")

In [ ]:
plot(SNN.vecplot(Post, :v),
xlabel = "Time (ms)", 
ylabel = "Membrane Potential (mV)", 
title = "Postsynaptic neuron with Poisson input")

In [ ]:
# Firing rates of the two neurons before connecting them
PreRate, PostRate = average_firing_rate(Pre, Post, 10000)
println("Firing rate presynaptic neuron: ", PreRate, "Hz, Firing rate postsynaptic neuron: ", PostRate, "Hz")

### After connecting neurons

In [ ]:
σPrePost = 1.27pF # Initial pre- to post-synaptic neuron weight
σPostPre = 48.7pF # Initial post- to pre-synaptic neuron weight
iSTDPparams = SNN.iSTDPParameter()
vSTDPparams = SNN.vSTDPParameter()

Pre = SNN.AdEx(; N = 1, param = LKD_AdEx_exc)
Post = SNN.IF(; N = 1, param = LKD_IF_inh) 

Input_Pre = SNN.Poisson(; N = 100, param = SNN.PoissonParameter(; rate = νpre))
ProjPre = SNN.SpikingSynapse(Input_Pre, Pre, :ge; σ = σ_in_E, p = p_in) # connection from input to presynaptic neuron
Input_Post = SNN.Poisson(; N = 100, param = SNN.PoissonParameter(; rate = νpost))
ProjPost = SNN.SpikingSynapse(Input_Post, Post, :ge; σ = σ_in_E, p = p_in) # connection from input to postsynaptic neuron

PrePost = SNN.SpikingSynapse(Pre, Post, :ge; σ = σPrePost, p = 1.0, param=vSTDPparams)
PostPre = SNN.SpikingSynapse(Post, Pre, :gi; σ = σPostPre, p = 1.0, param=iSTDPparams)

P = [Pre, Post, Input_Pre, Input_Post]
C = [PostPre, PrePost, ProjPre, ProjPost]

SNN.monitor([Pre, Post], [:v, :fire]) 
SNN.sim!(P, C; duration = duration)

In [ ]:
p1 = plot(SNN.vecplot(Pre, :v),
xlabel = "Time (ms)", 
ylabel = "Membrane Potential (mV)", 
title = "Presynaptic neuron with Poisson input")
p2 = plot(SNN.vecplot(Post, :v),
xlabel = "Time (ms)", 
ylabel = "Membrane Potential (mV)", 
title = "Postsynaptic neuron with Poisson input")
plot(p1, p2,  size=(1000, 450))

In [ ]:
# Firing rates of the two neurons before connecting them
PreRate, PostRate = average_firing_rate(Pre, Post, 10000)
println("Firing rate presynaptic neuron: ", PreRate, "Hz, Firing rate postsynaptic neuron: ", PostRate, "Hz")

### Varying initial weights

In [ ]:
iSTDPparams = SNN.iSTDPParameter()
vSTDPparams = SNN.vSTDPParameter()
νpre = 20Hz # Rate of external input to presynaptic neuron
νpost = 4.5Hz # Rate of external input to postsynaptic neuron

initial_weights_PrePost = 1pF:2pF:5pF
initial_weights_PostPre = 30pF:10pF:50pF

excitatory_list = []
inhibitory_list = []
for σPrePost in initial_weights_PrePost
    for σPostPre in initial_weights_PostPre
        println("σPrePost: ", σPrePost, ", σPostPre:", σPostPre)
        Pre = SNN.AdEx(; N = 1, param = LKD_AdEx_exc)
        Post = SNN.IF(; N = 1, param = LKD_IF_inh) 

        Input_Pre = SNN.Poisson(; N = 100, param = SNN.PoissonParameter(; rate = νpre))
        ProjPre = SNN.SpikingSynapse(Input_Pre, Pre, :ge; σ = σ_in_E, p = p_in) # connection from input to presynaptic neuron
        Input_Post = SNN.Poisson(; N = 100, param = SNN.PoissonParameter(; rate = νpost))
        ProjPost = SNN.SpikingSynapse(Input_Post, Post, :ge; σ = σ_in_E, p = p_in) # connection from input to postsynaptic neuron

        PrePost = SNN.SpikingSynapse(Pre, Post, :ge; σ = σPrePost, p = 1.0, param=vSTDPparams)
        PostPre = SNN.SpikingSynapse(Post, Pre, :gi; σ = σPostPre, p = 1.0, param=iSTDPparams)

        P = [Pre, Post, Input_Pre, Input_Post]
        C = [PostPre, PrePost, ProjPre, ProjPost]

        SNN.monitor([Pre, Post], [:v, :fire]) 
        W = zeros((2,1000)) # 1000 seconds, axis1: postpre, axis2: prepre
        start = 1
        ends = 0 
        spikes_per_sec_list = zeros(Int64, 1000)
        for i in 1:1000
            SNN.train!(P, C; duration = 1second)
            W[1,i] = PostPre.W[1] # mean(PostPre.W)
            W[2,i] = PrePost.W[1] # mean(PrePre.W)
            
            spikes_per_sec = sum(Pre.records[:fire][start:(ends*10000)+10000]) #    
            spikes_per_sec_list[i] =  spikes_per_sec[1]
            if i == 1
                start = 10000
            else
                start += 10000
            end
            ends += 1
        end
        push!(excitatory_list, W[2,:])
        push!(inhibitory_list, W[1,:])
    end
end 

# Create a figure with two subplots
p1 = plot(1:1:length(excitatory_list[1]), excitatory_list, title="Excitatory weight", label=false)
p2 = plot(1:1:length(inhibitory_list[1]), inhibitory_list, title="Inhibitory weight", label=false)
plot(p1, p2, layout=(2,1), size=(1000, 600), xlabel="Time (s)")

In [ ]:
# Create a figure with two subplots
p1 = plot(1:1:length(excitatory_list[1]), excitatory_list[1], title="Excitatory weight", label=false)
p2 = plot(1:1:length(inhibitory_list[1]), inhibitory_list[1], title="Inhibitory weight", label=false)
plot(p1, p2, layout=(2,1), size=(1000, 600), xlabel="Time (s)")

### Weights change over time

In [ ]:
σPrePost = 1.27pF # Initial pre- to post-synaptic neuron weight
σPostPre = 48.7pF # Initial post- to pre-synaptic neuron weight
iSTDPparams = SNN.iSTDPParameter()
vSTDPparams = SNN.vSTDPParameter()
νpre = 20Hz # Rate of external input to presynaptic neuron
νpost = 4.5Hz # Rate of external input to postsynaptic neuron

Pre = SNN.AdEx(; N = 1, param = LKD_AdEx_exc)
Post = SNN.IF(; N = 1, param = LKD_IF_inh) 

Input_Pre = SNN.Poisson(; N = 100, param = SNN.PoissonParameter(; rate = νpre))
ProjPre = SNN.SpikingSynapse(Input_Pre, Pre, :ge; σ = σ_in_E, p = p_in) # connection from input to presynaptic neuron
Input_Post = SNN.Poisson(; N = 100, param = SNN.PoissonParameter(; rate = νpost))
ProjPost = SNN.SpikingSynapse(Input_Post, Post, :ge; σ = σ_in_E, p = p_in) # connection from input to postsynaptic neuron

PrePost = SNN.SpikingSynapse(Pre, Post, :ge; σ = σPrePost, p = 1.0, param=vSTDPparams)
PostPre = SNN.SpikingSynapse(Post, Pre, :gi; σ = σPostPre, p = 1.0, param=iSTDPparams)

P = [Pre, Post, Input_Pre, Input_Post]
C = [PostPre, PrePost, ProjPre, ProjPost]

SNN.monitor([Pre, Post], [:v, :fire]) 

W = zeros((2,1000)) # 1000 seconds, axis1: postpre, axis2: prepre
start = 1
ends = 0 
spikes_per_sec_list = zeros(Int64, 1000)
for i in 1:1000
    SNN.train!(P, C; duration = 1second)
    W[1,i] = PostPre.W[1] # mean(PostPre.W)
    W[2,i] = PrePost.W[1] # mean(PrePre.W)
    spikes_per_sec = sum(Pre.records[:fire][start:(ends*10000)+10000]) #    
    spikes_per_sec_list[i] =  spikes_per_sec[1]
    if i == 1
        start = 10000
    else
        start += 10000
    end
    ends += 1
end
p1 = plot(W[2,:], label="Excitatory weight") # , ylims = (vSTDPparams.Wmin, vSTDPparams.Wmax)
p2 = plot(W[1,:], label="Inhibitory weight") # , ylims = (iSTDPparams.Wmin, iSTDPparams.Wmax)
plot(p1, p2,  size=(1000, 450), xlabel="Time (s)")

### Varying initial rate

In [ ]:
iSTDPparams = SNN.iSTDPParameter()
vSTDPparams = SNN.vSTDPParameter()
σPrePost = 1.27pF # Initial pre- to post-synaptic neuron weight
σPostPre = 48.7pF # Initial post- to pre-synaptic neuron weight
initial_rate_PrePost = 1Hz:5Hz:45Hz
initial_rate_PostPre = 1Hz:5Hz:45Hz

excitatory_list = []
inhibitory_list = []
for νpre in initial_rate_PrePost
    for νpost in initial_rate_PostPre
        println("in")
        Pre = SNN.AdEx(; N = 1, param = LKD_AdEx_exc)
        Post = SNN.IF(; N = 1, param = LKD_IF_inh) 

        Input_Pre = SNN.Poisson(; N = 100, param = SNN.PoissonParameter(; rate = νpre))
        ProjPre = SNN.SpikingSynapse(Input_Pre, Pre, :ge; σ = σ_in_E, p = p_in) # connection from input to presynaptic neuron
        Input_Post = SNN.Poisson(; N = 100, param = SNN.PoissonParameter(; rate = νpost))
        ProjPost = SNN.SpikingSynapse(Input_Post, Post, :ge; σ = σ_in_E, p = p_in) # connection from input to postsynaptic neuron

        PrePost = SNN.SpikingSynapse(Pre, Post, :ge; σ = σPrePost, p = 1.0, param=vSTDPparams)
        PostPre = SNN.SpikingSynapse(Post, Pre, :gi; σ = σPostPre, p = 1.0, param=iSTDPparams)

        P = [Pre, Post, Input_Pre, Input_Post]
        C = [PostPre, PrePost, ProjPre, ProjPost]

        SNN.monitor([Pre, Post], [:v, :fire]) 
        W = zeros((2,1000)) # 1000 seconds, axis1: postpre, axis2: prepre
        start = 1
        ends = 0 
        spikes_per_sec_list = zeros(Int64, 1000)
        for i in 1:1000
            SNN.train!(P, C; duration = 1second)
            W[1,i] = PostPre.W[1] # mean(PostPre.W)
            W[2,i] = PrePost.W[1] # mean(PrePre.W)
            
            spikes_per_sec = sum(Pre.records[:fire][start:(ends*10000)+10000]) #    
            spikes_per_sec_list[i] =  spikes_per_sec[1]
            if i == 1
                start = 10000
            else
                start += 10000
            end
            ends += 1
        end
        push!(excitatory_list, W[2,:])
        push!(inhibitory_list, W[1,:])
    end
end 

# Create a figure with two subplots
p1 = plot(1:1:length(excitatory_list[1]), excitatory_list, title="Excitatory weight", label=false)
p2 = plot(1:1:length(inhibitory_list[1]), inhibitory_list, title="Inhibitory weight", label=false)
plot(p1, p2, layout=(2,1), size=(1000, 600), xlabel="Time (s)")